In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import cv2,os
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D , BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-01-16 12:38:26.357197: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 12:38:26.397766: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-16 12:38:26.398250: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 12:38:27.304349: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Define the DAM layer
class DenseAssociativeMemoryLayer(layers.Layer):
    def __init__(self, num_classes, **kwargs):
        super(DenseAssociativeMemoryLayer, self).__init__(**kwargs)
        self.num_classes = num_classes

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                      shape=(input_shape[-1], self.num_classes),
                                      initializer="random_normal",
                                      trainable=True)
        super(DenseAssociativeMemoryLayer, self).build(input_shape)

    def call(self, inputs):
        # DAM operation as described in the paper
        inputs_normalized = tf.math.l2_normalize(inputs, axis=-1)
        kernel_normalized = tf.math.l2_normalize(self.kernel, axis=0)
        output = tf.matmul(inputs_normalized, kernel_normalized)
        output = tf.nn.softmax(output)
        return output

num_classes = 4
input_tensor = Input(shape=(224, 224, 3))
resnet_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = DenseAssociativeMemoryLayer(num_classes=num_classes)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs = resnet_model.input, outputs = output)

for layer in resnet_model.layers:
    layer.trainable = False
    
# Data generators
train_gen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rescale=1/255.)
val_gen = ImageDataGenerator(rescale=1/255.)
test_gen = ImageDataGenerator(rescale=1/255.)

train_dir = '/home/jayakumar/Documents/Re_ Resume/coffee_data/train'
val_dir = '/home/jayakumar/Documents/Re_ Resume/coffee_data/valid'
test_dir = '/home/jayakumar/Documents/Re_ Resume/coffee_data/test'

BATCH_SIZE = 32
SEED = 56
CLASS_MODE = 'categorical'
channels = 3
img_size = (224, 224)

train_flow_gen = train_gen.flow_from_directory(directory=train_dir,
                                              class_mode= CLASS_MODE,
                                              batch_size=BATCH_SIZE,
                                              target_size=img_size,
                                              seed=SEED)

val_flow_gen = val_gen.flow_from_directory(directory=val_dir,
                                            class_mode=CLASS_MODE,
                                            batch_size=BATCH_SIZE,
                                            target_size=img_size,
                                            seed=SEED)

test_flow_gen = test_gen.flow_from_directory(directory=test_dir,
                                            class_mode=CLASS_MODE,
                                            batch_size=BATCH_SIZE,
                                            target_size=img_size,
                                            seed=SEED)

# Compile the model
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['accuracy'])

rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6, mode='min', verbose=0)
early_cb = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=0)

# Train the model
model.fit(train_flow_gen, epochs=30,
          steps_per_epoch=int(np.ceil(train_flow_gen.samples / BATCH_SIZE)),
          validation_data=val_flow_gen,
          validation_steps=int(np.ceil(val_flow_gen.samples / BATCH_SIZE)),
          callbacks=[rlr_cb, early_cb])

# model.save("res_model.h5")

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_flow_gen)
print(f"Test Accuracy: {test_acc}")

Found 1200 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Found 400 images belonging to 4 classes.
Epoch 1/30


2024-01-16 12:42:58.851015: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


38/38 [==============================] - ETA: 0s - loss: 1.2989 - accuracy: 0.4200

2024-01-16 12:44:22.287679: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


38/38 [==============================] - 110s 3s/step - loss: 1.2989 - accuracy: 0.4200 - val_loss: 1.3297 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 2/30
38/38 [==============================] - 104s 3s/step - loss: 0.9682 - accuracy: 0.4908 - val_loss: 0.8916 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/30
38/38 [==============================] - 116s 3s/step - loss: 0.7999 - accuracy: 0.5000 - val_loss: 1.8479 - val_accuracy: 0.2500 - lr: 0.0010
Epoch 4/30
38/38 [==============================] - 115s 3s/step - loss: 0.7546 - accuracy: 0.5667 - val_loss: 0.7623 - val_accuracy: 0.4875 - lr: 0.0010
Epoch 5/30
38/38 [==============================] - 110s 3s/step - loss: 0.7024 - accuracy: 0.6317 - val_loss: 0.7105 - val_accuracy: 0.7150 - lr: 0.0010
Epoch 6/30
38/38 [==============================] - 103s 3s/step - loss: 0.6658 - accuracy: 0.6975 - val_loss: 0.6582 - val_accuracy: 0.7475 - lr: 0.0010
Epoch 7/30
38/38 [==============================] - 109s 3s/step - loss: 0.6243 - accur

2024-01-16 13:35:00.087673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


13/13 [==============================] - 28s 2s/step - loss: 0.2509 - accuracy: 0.9100
Test Accuracy: 0.9100000262260437
